# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""6616302572030853049""","""14050678190117924439""","""../../../../data/SampleHDS.jso…",9.9194e18,"""11513303761731358967""","""15355869157380624944""",false
"""15228677399548362036""","""12881372064907283909""","""../../../../data/SampleHDS.jso…",1.6889e19,"""11513303761731358967""","""4141251573738023459""",true
"""1011122717970142433""","""9080977037451594851""","""../../../../data/SampleHDS.jso…",3.5934e18,"""5623455175542837375""","""15355869157380624944""",false
"""15228677399548362036""","""6535481971343268908""","""../../../../data/SampleHDS.jso…",2.0975e17,"""7998931294576339294""","""15355869157380624944""",false
"""6616302572030853049""","""6971454181415477074""","""../../../../data/SampleHDS.jso…",1.8237e19,"""7998931294576339294""","""5160430314034407666""",true
"""1011122717970142433""","""18403429534744329775""","""../../../../data/SampleHDS.jso…",3.4962e18,"""2556055343863125234""",null,false
"""1011122717970142433""","""18073053886493952380""","""../../../../data/SampleHDS.jso…",3.0874e18,"""7998931294576339294""","""5160430314034407666""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""17324879526771560578""","""3392224152793663524""","""../../../../data/SampleHDS.jso…",1.7857e19,"""FirstMortgage30yr""","""13435076877075461193""","""Rejected"""
"""16107409146386019181""","""13768655868639185323""","""../../../../data/SampleHDS.jso…",9.8253e18,"""FirstMortgage30yr""","""16105933808583899482""","""Accepted"""
"""5353472830081425086""","""13677839057108814633""","""../../../../data/SampleHDS.jso…",5.6097e18,"""MoneyMarketSavingsAccount""","""13435076877075461193""","""Rejected"""
"""16107409146386019181""","""8777043500380242243""","""../../../../data/SampleHDS.jso…",1.0410e18,"""BasicChecking""","""13435076877075461193""","""Rejected"""
"""17324879526771560578""","""14634892853076604367""","""../../../../data/SampleHDS.jso…",3.2241e18,"""BasicChecking""","""16195302969627611179""","""Accepted"""
"""5353472830081425086""","""10249544514887979736""","""../../../../data/SampleHDS.jso…",3.5278e18,"""UPlusFinPersonal""",null,"""Rejected"""
"""5353472830081425086""","""5657954607517846942""","""../../../../data/SampleHDS.jso…",8.6557e17,"""BasicChecking""","""16195302969627611179""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""2305797378785213244""","""1993073785172396593""","""../../../../data/SampleHDS.jso…",1.4027e19,"""FirstMortgage30yr""","""3656759753578105976""","""Rejected"""
"""10075092616109179410""","""10408530804773147484""","""../../../../data/SampleHDS.jso…",8.2103e18,"""FirstMortgage30yr""","""18021867529932497997""","""Accepted"""
"""6634124755762446894""","""2656201801279481466""","""../../../../data/SampleHDS.jso…",9.0245e18,"""MoneyMarketSavingsAccount""","""3656759753578105976""","""Rejected"""
"""10075092616109179410""","""1333612034728154071""","""../../../../data/SampleHDS.jso…",1.1344e19,"""BasicChecking""","""3656759753578105976""","""Rejected"""
"""2305797378785213244""","""4202043386181501160""","""../../../../data/SampleHDS.jso…",3.2109e18,"""BasicChecking""","""1619020405048800549""","""Accepted"""
"""6634124755762446894""","""9548274274426951866""","""../../../../data/SampleHDS.jso…",1.3062e18,"""UPlusFinPersonal""",null,"""Rejected"""
"""6634124755762446894""","""11782253833193359036""","""../../../../data/SampleHDS.jso…",1.2956e19,"""BasicChecking""","""1619020405048800549""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""11775837397349681876""","""12835739524217674517""",1.4914e19,"""FirstMortgage30yr""","""14532148408731708026""","""Rejected"""
"""4827484455818938842""","""13769159894769691878""",1.7983e19,"""FirstMortgage30yr""","""7503806566214485707""","""Accepted"""
"""9575780208302168804""","""17245383806712465800""",7.8152e18,"""MoneyMarketSavingsAccount""","""14532148408731708026""","""Rejected"""
"""4827484455818938842""","""18026833732185522692""",1.1669e19,"""BasicChecking""","""14532148408731708026""","""Rejected"""
"""11775837397349681876""","""4524813780908682509""",9.5927e18,"""BasicChecking""","""1365725536712826993""","""Accepted"""
"""9575780208302168804""","""12347376102245873313""",6.2754e18,"""UPlusFinPersonal""",null,"""Rejected"""
"""9575780208302168804""","""2747428689959226259""",4.1345e18,"""BasicChecking""","""1365725536712826993""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'